# Building a data analysis company with Group Chat

In [1]:
## import required libraries
import os
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager
from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv(override=True) # load environment variables from .env file

True

In [2]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## The task!

In [3]:
task = "Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief report."

## Create a group chat

This group chat will include these agents:

- User_proxy or Admin: to allow the user to comment on the report and ask the writer to refine it.
- Planner: to determine relevant information needed to complete the task.
- Engineer: to write code using the defined plan by the planner.
- Executor: to execute the code written by the engineer.
- Writer: to write the report.

In [4]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="Give the task, and send instructions to the writer to refine the stock analysis report.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS"
)

In [5]:
## planner agent
planner = ConversableAgent(
    name="Planner",
    system_message="""
    Given a task, please determine what stock information is needed to complete the task. 
    Please note that the information will all be retrieved using Python code. 
    Please only suggest information that can be retrieved using Python code. 
    After each step is done by others, check the progress and instruct the remaining steps. 
    If a step fails, try to workaround.
    """,
    llm_config=llm_config,
)

In [6]:
## engineer agent
engineer = AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code to fetch and analyze stock data based on the plan provided by the planner.",
)

In [7]:
## executor agent
executor = ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the engineer and report the stock data results.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 6,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [8]:
## writer agent
writer = ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Stock Analysis Report Writer. Please write stock analysis reports in markdown format (with relevant titles) and put the content in pseudo ```md``` code block. You take feedback from the admin and refine your report.",
    description="Writer. Write stock analysis reports based on the code execution results and take feedback from the admin to refine the report."
)

## Define the group chat

In [9]:
groupchat = GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [10]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

## Start the group chat

In [11]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task
)

Admin (to chat_manager):

Analyze the daily closing prices of Apple (AAPL) stock for the past 1 month and create a brief report.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To complete the task of analyzing the daily closing prices of Apple (AAPL) stock for the past 1 month and creating a brief report, here is the list of stock information needed and the steps to follow:

### Step 1: Retrieve Stock Data for the Past 1 Month
**Required Information:**
- Ticker symbol: AAPL
- Time period: Past 1 month
- Data type: Daily closing prices

**Python Packages to Use:**
- `yfinance` or `pandas_datareader` for data retrieval
- `pandas` for data manipulation
- `matplotlib` or `seaborn` for visualization (optional, for the report)

**Sample Code:**
```python
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the ticker symbol
ticker = 'AAP

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):

It seems there was an issue with running the provided script. Let's reattempt the execution with a slightly refined approach, ensuring the code is complete and executable.

Here is the corrected and complete Python script for performing data analysis:

```python
# filename: analyze_aapl_data.py
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Step 1: Retrieve Stock Data
# Define the ticker symbol for Apple
ticker = 'AAPL'

# Define the time period (past 1 month)
end_date = datetime.today()
start_date = end_date - timedelta(days=30)

# Retrieve stock data using yfinance
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Extract daily closing prices
closing_prices = stock_data['Close']

# Step 2: Perform Data Analysis

# Calculate summary statistics
summary_stats = closing_prices.describe()

# Calculate daily returns and assess volatility
daily_